<a href="https://colab.research.google.com/github/jioffe502/kmerkounters/blob/main/kmercount.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#install mrjob
!pip install mrjob

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 439.6/439.6 kB 6.1 MB/s eta 0:00:00


In [ ]:
!pip install boto3

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.6/135.6 kB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.6/10.6 MB 64.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.6/79.6 kB 9.2 MB/s eta 0:00:00


In [ ]:
!apt-get install p7zip-full

Reading package lists... Done
Building dependency tree       
Reading state information... Done
p7zip-full is already the newest version (16.02+dfsg-7build1).
0 upgraded, 0 newly installed, 0 to remove and 24 not upgraded.


In [ ]:
!pip install jar

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
ERROR: Could not find a version that satisfies the requirement jar (from versions: none)
ERROR: No matching distribution found for jar


In [ ]:
from google.colab import files

# upload dataset
uploaded = files.upload()
for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

Saving ioffe.pem to ioffe.pem
User uploaded file "ioffe.pem" with length 1674 bytes


In [ ]:
from google.colab import files

# upload mrjob.conf under current directory(/content/mrjob.conf)
uploaded = files.upload()
for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

Saving mrjob.conf to mrjob.conf
User uploaded file "mrjob.conf" with length 651 bytes


In [ ]:
!chmod 400 ioffe.pem

In [ ]:
!7z x chr1.zip


7-Zip [64] 16.02 : Copyright (c) 1999-2016 Igor Pavlov : 2016-05-21
p7zip Version 16.02 (locale=en_US.UTF-8,Utf16=on,HugeFiles=on,64 bits,2 CPUs Intel(R) Xeon(R) CPU @ 2.20GHz (406F0),ASM,AES-NI)

Scanning the drive for archives:
  0M Scan         1 file, 1077936128 bytes (1028 MiB)

Extracting archive: chr1.zip

ERRORS:
Unexpected end of archive

--
Path = chr1.zip
Type = zip
ERRORS:
Unexpected end of archive
Physical Size = 1350864766

  0%      0% - NA24385_illumina_hg19_chr1.fa                                      1% - NA24385_illumina_hg19_chr1.fa                                      2% - NA24385_illumina_hg19_chr1.fa                                      3% - NA24385_illumina_hg19_chr1.fa                   

In [ ]:
%%file kmercount.py
from mrjob.job import MRJob
import json
import re


'''
Count the number of kmers.
'''

class KmerCount(MRJob):

    def mapper(self, _, line):
        # Parse the FASTA file and extract the sequence
        if line.startswith(">"):
            seq = ""
        else:
            seq = line.strip()
            # Count the number of kmers of size 1
            for i in range(len(seq)):
                kmer = seq[i:i+1]
                yield kmer, 1
    
    def reducer(self, kmer, counts):
        # Sum the counts for each kmer
        yield kmer, sum(counts)


if __name__ == '__main__':
    KmerCount.run()

Overwriting kmercount.py


In [ ]:
%%file kmercount.py
from mrjob.job import MRJob
import json
import re


'''
Count the number of kmers.
'''

class KmerCount(MRJob):

    def configure_args(self):
        super(KmerCount, self).configure_args()
        self.add_passthru_arg('--kmer-length', type=int, default=1, help='Length of the kmer')

    def mapper(self, _, line):
        # Parse the FASTA file and extract the sequence
        if line.startswith(">"):
            seq = ""
        else:
            seq = line.strip()
            kmer_length = self.options.kmer_length
            # Count the number of kmers of specified size
            for i in range(len(seq) - kmer_length + 1):
                kmer = seq[i:i+kmer_length]
                yield kmer, 1

    
    def reducer(self, kmer, counts):
        # Sum the counts for each kmer
        yield kmer, sum(counts)
    
    combiner = reducer



if __name__ == '__main__':
    KmerCount.run()


Overwriting kmercount.py


In [ ]:
!zip sample.zip /content/chr1sample.zip

  adding: content/chr1sample.zip


zip error: Interrupted (aborting)


In [ ]:
%%time
!python kmercount.py --kmer-length 3 -r local /content/NA24385_illumina_hg19_chr1_first1000.fa > kmercount.out

No configs found; falling back on auto-configuration
No configs specified for local runner
Creating temp directory /tmp/kmercount.root.20230426.054540.596454
Running step 1 of 1...
job output is in /tmp/kmercount.root.20230426.054540.596454/output
Streaming final output from /tmp/kmercount.root.20230426.054540.596454/output...
Removing temp directory /tmp/kmercount.root.20230426.054540.596454...
CPU times: user 31.6 ms, sys: 14.9 ms, total: 46.5 ms
Wall time: 3.45 s


In [17]:
#run the program. Capture output
%%time
!python kmercount.py -r emr s3://vandy-bigdata-kmercount/output.fa --cloud-tmp-dir=s3://vandy-bigdata-kmercount/tmp --cluster-id=j-15LBG95BYQVWM --conf-path /content/mrjob.conf > kmercountfinal.out

Creating temp directory /tmp/kmercount.root.20230426.060255.072131
uploading working dir files to s3://vandy-bigdata-kmercount/tmp/kmercount.root.20230426.060255.072131/files/wd...
Copying other local files to s3://vandy-bigdata-kmercount/tmp/kmercount.root.20230426.060255.072131/files/
Adding our job to existing cluster j-15LBG95BYQVWM
  master node is ec2-75-101-175-35.compute-1.amazonaws.com
Waiting for Step 1 of 1 (s-1BLU72OJLTY34) to complete...
  PENDING (cluster is RUNNING: Running step)
  PENDING (cluster is RUNNING: Running step)
  PENDING (cluster is RUNNING: Running step)
  PENDING (cluster is RUNNING: Running step)
  PENDING (cluster is RUNNING: Running step)
  RUNNING for 0:00:37
    10.0% complete
  RUNNING for 0:01:08
    10.0% complete
  RUNNING for 0:01:39
    10.0% complete
  RUNNING for 0:02:10
    10.1% complete
  RUNNING for 0:02:40
    10.1% complete
  RUNNING for 0:03:11
    10.2% complete
  RUNNING for 0:03:42
    10.2% complete
Traceback (most recent call last)